<a href="https://colab.research.google.com/github/Hidetsune/SemEval2024_Task10/blob/main/SemEval2024_Task10_main_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Let's get started!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

In [ ]:
import re

In [ ]:
import spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")


2024-01-14 05:02:38.775231: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 05:02:38.775293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 05:02:38.776902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 05:02:40.499140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


#Datasets
[Reference](https://heartbeat.comet.ml/emotion-classification-with-spacy-v3-comet-eaff310c0d7c)

In [ ]:
!pip install "spacy >= 3.0.6"
!pip install "comet_ml>=3.31.19"
!pip install datasets

  Using cached comet_ml-3.36.0-py3-none-any.whl (589 kB)
  Using cached python_box-6.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached sentry_sdk-1.39.2-py2.py3-none-any.whl (254 kB)
  Using cached simplejson-3.19.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (137 kB)
  Using cached websocket_client-1.3.3-py3-none-any.whl (54 kB)
  Using cached wurlitzer-3.0.3-py3-none-any.whl (7.3 kB)
  Using cached everett-3.1.0-py2.py3-none-any.whl (35 kB)
  Using cached dulwich-0.21.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (514 kB)
  Using cached configobj-5.0.8-py2.py3-none-any.whl (36 kB)
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled

In [ ]:
%env COMET_API_KEY=<"zKfFM345o2F1kk8KGHtVJy7kO">
%env COMET_PROJECT_NAME=<"semeval2024-Task3">

env: COMET_API_KEY=<"zKfFM345o2F1kk8KGHtVJy7kO">
env: COMET_PROJECT_NAME=<"semeval2024-Task3">


In [ ]:
text_emotion_pair = pd.read_csv("/content/drive/MyDrive/SemEval Task10/df_train_val_ni3000_ver1.csv") #CHANGE HERE
text_emotion_pair

,emotions,utterances_English
0,neutral,Look whether Maya is speaking or not! Then you...
1,neutral,"Indravadan, for how many years have you been m..."
2,neutral,"Maya, Monisha and Sahil are coming, see what I..."
3,neutral,"Hee darling. Well Indravadan, please or throw ..."
4,neutral,"Ok Maya, come on Rosesh throw you out"
...,...,...
19621,fear,bar home looks like wedding reception
19622,fear,Good nervous making permanent change
19623,fear,walked past Asian girl grabs BFs arm know race...
19624,fear,hate calling people


In [ ]:
text_emotion_pair.emotions.unique()

array(['neutral', 'contempt', 'surprise', 'anger', 'sadness', 'joy',
       'disgust', 'fear'], dtype=object)

In [ ]:
categories = ['neutral', 'contempt', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
text_emotion_pair_train, text_emotion_pair_validation_evaluation = train_test_split(text_emotion_pair, test_size=0.05, random_state=10)
text_emotion_pair_validation, text_emotion_pair_evaluation = train_test_split(text_emotion_pair_validation_evaluation, test_size=0.001, random_state=101)

In [ ]:
from datasets import Dataset, Value, ClassLabel, Features  #added

In [ ]:
from datasets import Dataset
dataset_text_emotion_train = Dataset.from_pandas(text_emotion_pair_train)
dataset_text_emotion_validation = Dataset.from_pandas(text_emotion_pair_validation)
dataset_text_emotion_evaluation = Dataset.from_pandas(text_emotion_pair_evaluation)

In [ ]:
new_features = dataset_text_emotion_train.features.copy()

In [ ]:
new_features['utterances_English'] = Value('large_string')
new_features['emotions'] = ClassLabel(names=['neutral', 'contempt', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear'])

In [ ]:
dataset_text_emotion_train = dataset_text_emotion_train.cast(new_features)
dataset_text_emotion_validation = dataset_text_emotion_validation.cast(new_features)
dataset_text_emotion_evaluation = dataset_text_emotion_evaluation.cast(new_features)

Casting the dataset:   0%|          | 0/18644 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/981 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
dataset_text_emotion_train.features

{'emotions': ClassLabel(names=['neutral', 'contempt', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear'], id=None),
 'utterances_English': Value(dtype='large_string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
dataset_text_emotion_validation.features

{'emotions': ClassLabel(names=['neutral', 'contempt', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear'], id=None),
 'utterances_English': Value(dtype='large_string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [ ]:
dataset_text_emotion_evaluation.features

{'emotions': ClassLabel(names=['neutral', 'contempt', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear'], id=None),
 'utterances_English': Value(dtype='large_string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

#Training

In [ ]:
from typing import Set, List, Tuple
from spacy.tokens import DocBin
import spacy
from tqdm.auto import tqdm

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding

In [ ]:
def convert_spacy_dataset(dataset, target_file: str, labels):
  nlp = spacy.blank("en")
  db = DocBin()

  for item in tqdm(dataset):
    doc = nlp.make_doc(item["utterances_English"])
    doc.cats = {label: 0 for label in labels}
    doc.cats[labels[item["emotions"]]] = 1

    db.add(doc)

  db.to_disk(target_file)
  return db

In [ ]:
convert_spacy_dataset(dataset_text_emotion_train, "train_data.spacy", categories)
convert_spacy_dataset(dataset_text_emotion_validation, "validation_data.spacy", categories)
convert_spacy_dataset(dataset_text_emotion_evaluation, "test_data.spacy", categories)

  0%|          | 0/18644 [00:00<?, ?it/s]

  0%|          | 0/981 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!python -m spacy init config --lang en --pipeline textcat_multilabel comet_config.cfg  #training model

2024-01-14 05:05:36.452647: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 05:05:36.452708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 05:05:36.454454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 05:05:37.785256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- 

In [ ]:
from spacy.cli.train import train as spacy_train

config_path = "comet_config.cfg"
output_model_path = "output_models/"

spacy_train(
    config_path,
    output_path=output_model_path,
    overrides={
        "paths.train": "train_data.spacy",
        "paths.dev": "validation_data.spacy",
    },
    use_gpu=0

)


✔ Created output directory: output_models
ℹ Saving to output directory: output_models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       48.99    0.49
  0     200          36.83       56.88    0.57
  0     400          26.17       59.67    0.60
  0     600          22.83       62.11    0.62
  0     800          21.08       64.76    0.65
  0    1000          20.08       66.44    0.66
  1    1200          18.88       67.90    0.68
  1    1400          17.32       69.23    0.69
  1    1600          17.00       70.60    0.71
  2    1800          16.10       71.86    0.72
  3    2000          15.30       72.93    0.73
  3    2200          14.44       74.31    0.

In [ ]:
#We load our spaCy model from disk
trained_nlp = spacy.load("output_models/model-best/")


texts = ['im feeling pretty anxious',
        'i don t feel particularly agitated',
        'i find myself in the odd position of feeling supportive of',
        'i feel so cold a href http irish',
        'im feeling very peaceful about our wedding again now after having'
        ]

category_scores = [doc.cats for doc in trained_nlp.pipe(texts)]


thresh = 0.2
for d in category_scores:
  print(dict((k, v) for k, v in d.items() if v >= thresh))

{'disgust': 0.2043335884809494, 'fear': 0.3514052927494049}
{'neutral': 0.3059558868408203, 'fear': 0.6084080934524536}
{'sadness': 0.37262392044067383, 'joy': 0.3439670503139496}
{'joy': 0.30930447578430176}
{'sadness': 0.26154208183288574, 'joy': 0.3772801458835602}


In [ ]:
text_emotion_pair_evaluation.emotion_list = list(text_emotion_pair_evaluation.emotions)
text_emotion_pair_evaluation.text_list = list(text_emotion_pair_evaluation.utterances_English)

<ipython-input-47-ce93554364ec>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  text_emotion_pair_evaluation.emotion_list = list(text_emotion_pair_evaluation.emotions)
<ipython-input-47-ce93554364ec>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  text_emotion_pair_evaluation.text_list = list(text_emotion_pair_evaluation.utterances_English)


In [ ]:
trained_nlp = spacy.load("output_models/model-best/")


texts = text_emotion_pair_evaluation.text_list[0:6]

category_scores = [doc.cats for doc in trained_nlp.pipe(texts)]


thresh = 0.1
for d in category_scores:
  print(dict((k, v) for k, v in d.items() if v >= thresh))

{'neutral': 0.30956652760505676, 'contempt': 0.11448843777179718, 'surprise': 0.34482643008232117, 'anger': 0.16035394370555878, 'sadness': 0.27392154932022095, 'joy': 0.2629300355911255, 'disgust': 0.14583207666873932, 'fear': 0.1675647348165512}


#Trial (Eval Data)

In [ ]:
Trial_Data = pd.read_csv("/content/drive/MyDrive/SemEval Task10/SemEval2024_Task10_subtask1_testset(eval)_translated.csv") #CHANGE HERE

In [ ]:
Trial_Data

,episode,utterances,speakers,utterances_English
0,roseshantstobeadopted.wmv,kuchh karo sahil please kuchh karo. mera roses...,maya,Please go to the beach. My rosesh adopted me a...
1,roseshantstobeadopted.wmv,monisha tum muskura kya rahi ho?,indu,Monisha why are you smiling?
2,roseshantstobeadopted.wmv,haila! main? main kahan? kya daddy ji?,monisha,Shake! I? where am i? What daddy?
3,roseshantstobeadopted.wmv,maya kuchh actually marne wali nahin hai. it's...,indu,Maya Kuchh is not actually going to die. it's ...
4,roseshantstobeadopted.wmv,stop it! stop it! stop it! tumhari isi tarah k...,maya,Stop it! Stop it! Stop it! Because of your act...
...,...,...,...,...
1575,prefinale,"99, 100. haan bade mandir se pure 100 kadam do...",rosesh,"99, 100. Yes, I was 100 steps away from the te..."
1576,prefinale,"haan medium kadam. itne. ohh, matlab 1.5 foot....",rosesh,"Yes, moderate steps. So much. Ohhh, that means..."
1577,prefinale,"1, 2, 3, 4, 5. woopie, mere right me chota man...",rosesh,"1, 2, 3, 4, 5. Woopa, there's a small temple t..."
1578,prefinale,pakka aegi na meri jeevan sathi? mujhe 51 year...,rosesh,Will my life partner definitely come? I don't ...


In [ ]:
Trial_Data.iloc[0].utterances

'kuchh karo sahil please kuchh karo. mera rosesh adopt ho karke chala gaya na to me, i know this sounds horribly melodramatic, monishaish, par me mar jaaungi. i swear mein mar jaaungi'

In [ ]:
Trial_Data["Pred_Emotion"] = Trial_Data["utterances_English"].apply(lambda x: max(trained_nlp(x).cats, key =trained_nlp(x).cats.get ))

In [ ]:
Trial_Data

,episode,utterances,speakers,utterances_English,Pred_Emotion
0,roseshantstobeadopted.wmv,kuchh karo sahil please kuchh karo. mera roses...,maya,Please go to the beach. My rosesh adopted me a...,fear
1,roseshantstobeadopted.wmv,monisha tum muskura kya rahi ho?,indu,Monisha why are you smiling?,neutral
2,roseshantstobeadopted.wmv,haila! main? main kahan? kya daddy ji?,monisha,Shake! I? where am i? What daddy?,surprise
3,roseshantstobeadopted.wmv,maya kuchh actually marne wali nahin hai. it's...,indu,Maya Kuchh is not actually going to die. it's ...,neutral
4,roseshantstobeadopted.wmv,stop it! stop it! stop it! tumhari isi tarah k...,maya,Stop it! Stop it! Stop it! Because of your act...,anger
...,...,...,...,...,...
1575,prefinale,"99, 100. haan bade mandir se pure 100 kadam do...",rosesh,"99, 100. Yes, I was 100 steps away from the te...",joy
1576,prefinale,"haan medium kadam. itne. ohh, matlab 1.5 foot....",rosesh,"Yes, moderate steps. So much. Ohhh, that means...",neutral
1577,prefinale,"1, 2, 3, 4, 5. woopie, mere right me chota man...",rosesh,"1, 2, 3, 4, 5. Woopa, there's a small temple t...",joy
1578,prefinale,pakka aegi na meri jeevan sathi? mujhe 51 year...,rosesh,Will my life partner definitely come? I don't ...,joy


In [ ]:
Tryal_Data2 = Trial_Data["Pred_Emotion"]

In [ ]:
for i in range(1580,17912):
  Tryal_Data2.loc[i] = "0.0"

In [ ]:
Tryal_Data2

0            fear
1         neutral
2        surprise
3         neutral
4           anger
           ...   
17907         0.0
17908         0.0
17909         0.0
17910         0.0
17911         0.0
Name: Pred_Emotion, Length: 17912, dtype: object

In [ ]:
Tryal_Data2.to_string("answer.txt", header=False, index=False)